# Ритейл — Анализ программы лояльности

## Задачи проекта:

Проанализировать программу лояльности магазина.

Провести исследовательский анализ данных;\
Провести анализ программы лояльности;\
Сформулировать и проверить статистические гипотезы.\
Проверить гипотезу: средний чек участников программы лояльности выше, чем у остальных покупателей.\
Сформулировать собственную ретроспективную статистическую гипотезу. Дополнить ее нулевой и альтернативной гипотезами.\
Проверить гипотезы с помощью статистического теста.

## Описание данных:
Датасет содержит данные о покупках в магазине строительных материалов «Строили, строили и наконец построили». \
Все покупатели могут получить доступ в магазин с использованием персональных карт. \
За 200 рублей в месяц они могут стать участниками программы лояльности. \
В программу включены скидки, специальные предложения, подарки.

В нашем распоряжении два датасета:\
retail_dataset.csv - журнал продаж\
product_codes.csv - стоимость товара

retail_dataset.csv:

- `purchaseId` — id чека;
- `item_ID` — id товара;
- `purchasedate` — дата покупки;
- `Quantity` — количество товара;
- `CustomerID` — id покупателя;
- `ShopID` — id магазина;
- `loyalty_program` — участвует ли покупатель в программе лояльности;

product_codes.csv:

- `productID` — id товара;
- `price_per_one` — стоимость одной единицы товара;

### Стадии проекта:
предобработка данных;\
исследовательский анализ данных;\
проверка гипотез;\
работа с метриками и бизнес-показателями;\
подготовка презентации.

### Поведенческие алгоритмы
прояснение запроса;\
декомпозиция задачи;\
работа с документацией,\
презентация результатов,\
и другие.

In [34]:
#загрузим нужные библиотеки
import pandas as pd
import numpy as np
from numpy import mean
import math as mth
import datetime as dt
import seaborn as sns
import scipy.stats as st
import plotly.express as px
import matplotlib.pyplot as plt
from plotly import graph_objects as go

In [48]:
#загрузим данные
product, retail = (
pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\product_codes.csv", sep=','), #стоимость товара
pd.read_csv('C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\retail_dataset.csv', sep=',') #журнал продаж
)
  

In [36]:
#общая информация о наборе данных в журнале продаж
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105335 entries, 0 to 105334
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   purchaseid       105335 non-null  object 
 1   item_ID          105335 non-null  object 
 2   Quantity         105335 non-null  int64  
 3   purchasedate     105335 non-null  object 
 4   CustomerID       69125 non-null   float64
 5   ShopID           105335 non-null  object 
 6   loyalty_program  105335 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ MB


В датасете  105335 записей, 7 столбцов. Три типа данных: object, float64 и int64. В столбце 'CustomerID' есть пропущенные значения. 

In [50]:
#количество пропусков в столбце CustomerID
retail['CustomerID'].isna().sum()

36210

In [51]:
#переименуем названия столбцов
retail = retail.rename(
   columns={
       'purchaseid': 'purchase_id',\
       'item_ID': 'item_id',\
       'Quantity': 'quantity',\
       'purchasedate': 'purchase_date',\
       'CustomerID': 'customer_id',\
       'ShopID': 'shop_id'\
   }
)

In [52]:
retail

,purchase_id,item_id,quantity,purchase_date,customer_id,shop_id,loyalty_program
0,538280,21873,11,2016-12-10 12:50:00,18427.0,Shop 0,0.0
1,538862,22195,0,2016-12-14 14:11:00,22389.0,Shop 0,1.0
2,538855,21239,7,2016-12-14 13:50:00,22182.0,Shop 0,1.0
3,543543,22271,0,2017-02-09 15:33:00,23522.0,Shop 0,1.0
4,543812,79321,0,2017-02-13 14:40:00,23151.0,Shop 0,1.0
...,...,...,...,...,...,...,...
105330,538566,21826,1,2016-12-13 11:21:00,NaN,Shop 0,0.0
105331,540247,21742,0,2017-01-05 15:56:00,21143.0,Shop 0,0.0
105332,538068,85048,1,2016-12-09 14:05:00,23657.0,Shop 0,1.0
105333,538207,22818,11,2016-12-10 11:33:00,18427.0,Shop 0,0.0


In [ ]:
# преобразование данных о времени
#costs['dt'] = pd.to_datetime(costs['dt']).dt.date

#Год выхода игр 'year_of_release' и оценки критиков 'critic_score' переведем из вещественного типа в целочисленный
#data['year_of_release'] = data['year_of_release'].astype('int')
#data['critic_score'] = data['critic_score'].astype('int')

#и приведем к вещественному типу 'object'
#data['user_score'] = data['user_score'].astype('float')

#удалим из самого датафрейма строки с явными дубликатами, оставив только первые вхождения: будем считать, что верные идентификаторы встречаются первыми
#data = data.drop_duplicates(subset=['name','platform'], keep='first')

In [54]:
#проверим на явные дубликаты
retail.duplicated().sum()


1033

In [65]:
#количество клиентов участвующих в программе лояльности(1.0)
retail['loyalty_program'].value_counts()

0.0    81493
1.0    23842
Name: loyalty_program, dtype: int64

In [66]:
#выведем количество уникальных пользователей по группе лояльности
retail.groupby('loyalty_program').agg({'customer_id':'nunique'})

,customer_id
loyalty_program,
0.0,1162
1.0,587


In [68]:
retail['customer_id'].unique().sum()

nan

In [57]:
#
retail.groupby('shop_id').agg({'purchase_id':'count'}).sort_values(by='purchase_id', ascending=False).head()


,purchase_id
shop_id,
Shop 0,97393
Shop 4,1741
Shop 1,1540
Shop 6,1032
Shop 8,560


In [70]:
#
retail.groupby('shop_id').agg({'customer_id':'nunique'}).sort_values(by='customer_id', ascending=False).head()

,customer_id
shop_id,
Shop 0,1572
Shop 4,43
Shop 1,38
Shop 8,12
Shop 12,10


In [42]:
#общая информация о наборе данных
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9969 entries, 0 to 9968
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   productID      9969 non-null   object 
 1   price_per_one  9969 non-null   float64
dtypes: float64(1), object(1)
memory usage: 155.9+ KB


In [43]:
#переименуем название столбца
product = product.rename(columns={'productID': 'product_id'})

In [44]:
product

,product_id,price_per_one
0,85123A,2.55
1,71053,3.39
2,84406B,2.75
3,84029G,3.39
4,84029E,3.39
...,...,...
9964,84306,8.29
9965,90001A,4.15
9966,90118,2.46
9967,DOT,172.62


# Предобработка данных

Исследовать пропущенные значения;\
Исследовать соответствие типов;\
Исследовать дубликаты;\
Проверить корректность наименований колонок;\
Переименовать колонки;\
Удалить дубликаты;\
Привести типы;\
Заменить пропущенные значения.

### Исследуем пропущенные значения

# Исследовательский анализ данных